In [ ]:
import json
import pickle
import re
import os
import numpy as np
from numpy import ma
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal as sig
import pandas as pd
import yaml
from pykalman import KalmanFilter
from timeseries import SampledTimeSeries
from mipcat.signal.timeseries_generator import ts_from_pickle
%matplotlib notebook

# Folder with dataset

Change this to the folder where the downloaded dataset (`mozart_sample.zip`) was extracted and the folder where calculated timeseries reside

In [ ]:
dataroot = "/mipcat/data/original/"
calcroot = "/mipcat/data/calc"

# Read segmentation

`mozart_notes.csv` contains a compilation of the segmentations for all the recordings in the dataset

In [ ]:
notedf = pd.read_csv(os.path.join(calcroot,"mozart_notes.csv"),index_col=None)

# Add metadata

Extract some needed information, some contained in the filenames, other from the music sheet

In [ ]:

#notedf = notedf[notedf['clip'].isin(['1', '2', 'mozart (0)'])]

# Convert windows paths to unix paths
notedf["text_grid"]=notedf.text_grid.str.replace("\\","/",regex=False)

# Note index is contained in the segmentation label
# This corresponds to the order number of the note in the music sheet
notedf['note_idx'] = notedf.label.str.extract('^([0-9]+).*').astype('int')

# Generate a unique index number for each version played
notedf['gl_ver_idx'] = notedf.groupby(['text_grid','clip'])[['text_grid','clip']].ngroup()

# Gather the subject number from the filename
notedf['subj_id'] = notedf.text_grid.apply(lambda x: x.split('/')[-3])

# Gather some more information from the file path
subdirs = notedf.text_grid.apply(lambda x: x.split('/')[-2])
notedf['instrument'] = 'lab'
notedf.loc[notedf.text_grid.str.contains('Own'),'instrument']='own'

notedf["sigfile"]=notedf.text_grid.str.replace("_notes.TextGrid","_ts.pickle",regex=False)
#notedf["duration"]=notedf.end-notedf.start



In [ ]:
with open(dataroot+"melodies.yaml") as f:
    meljs = yaml.safe_load(f)
    
tunejs = meljs['mozart']
tunedf = pd.DataFrame(tunejs['notes'])
tune_bar = tunejs['bar_duration']

b = 0
for ii, n in tunedf.iterrows():
    if n.strong_beat==True:
        b=0
    if b%tune_bar==0:
        tunedf.loc[ii,'strong_beat'] = True
    b+=n.duration
    
tunedf['start_beat']=tunedf.shift().duration.cumsum().fillna(0)
tunedf = tunedf[tunedf.pitch!=0].reset_index()
#tunedf.loc[tunedf.pitch!=0,'note_idx'] = np.arange(sum(tunedf.pitch!=0))

ndf = notedf.join(tunedf,on='note_idx',how='left')

# get name of original wav file
# needed to get video file
wavdf = pd.read_csv(dataroot+'/wav_list.csv',index_col=0)
wm=ndf.text_grid.apply(lambda x:pd.Series([x.find(os.path.splitext(y)[0]) for y in wavdf.filename],index=wavdf.filename))
ndf['wavfile']=wm.idxmax(axis=1)
ndf[(wm<0).all(axis=1)] = ''

In [ ]:
ndf.groupby('gl_ver_idx').apply(lambda x: x[['subj_id','instrument']].iloc[0])

# Read video information

The following file contains information such as video delay relative to main signal file, and clainet marker IDs

In [ ]:
videodf = pd.read_csv(dataroot+'/wav_video_delays.csv')
videodf

# Choose trials to compare

Write the global version numbers next (`gl_ver_idx` column in the notedf table)

In [ ]:
trials = [1,2,4,5];

# Parse clarinet marker file

Reads the marker file and chooses the relevant markers

In [ ]:
def parse_aruco(data):
    times = np.zeros(len(data))
    aruco_ids = np.array(list(set([int(x) for y in data for x in y  if re.match(r'^[0-9]+$',x)])))
    id_max = len(aruco_ids)

    aruco_arr = np.ones((len(data),len(aruco_ids),2))*np.nan
    aruco_dim = np.ones((len(data),len(aruco_ids),2))*np.nan

    for fno, ad in enumerate(data):
        times[fno] = ad['msec']/1000
        for ii, aid in enumerate(aruco_ids):
            try:
                val = ad[str(aid)]
            except KeyError:
                continue
            rect = val['bbox']
            y = -(rect[1]+rect[3]/2)
            x = (rect[0]+rect[2]/2)
            aruco_dim[fno, ii, :] = [rect[2],rect[3]]
            aruco_arr[fno, ii, :] = [x,y]
        times[fno] = ad['msec'] / 1000
        
    return aruco_ids, times, aruco_arr, aruco_dim




# Kalman filtering

Custom Kalman filter for 2D motion that tries to remove outliers

In [ ]:
def kfilter(x, observation_error=0.05, observation_cov=0.001,
               pos_proc_error=0.15, pos_proc_xy_cov=0.015,
               vel_proc_error=0.25, vel_proc_xy_cov=0.05,
               pos_vel_proc_xx_cov=0.1, pos_vel_proc_xy_cov=0.025,
               outl_cov_fact=100000, outl_pos_thresh=50):
    """
    Use Kalman to filter a XY position sequence.
    
    observation_error and observation_cov is the estimated error/cov of the measured position
    the 6 _proc_error and _proc_cov are coefficients for the covariance matrix of the model 
    (model is x_ij+1 = x_ij+v_ij*t)
    outl_cov_fact is a factor applied to process (model) transition matrix. Matrix is divided by this factor,
    making for a more aggressive filtering
    Outliers that move further than outl_pos_thresh of the filtered model are treated as missing observations
    """
    preds = np.zeros(x.shape)
    for ii in range(x.shape[1]):
        # Find nan values 
        meas = ma.array(x[:,ii,:].copy())
        naidx = np.flatnonzero(np.isnan(meas[:,0]))
        ndiff = np.diff(naidx)

        # Calculate initial state estimate based on non-nan values
        if len(ndiff)>0 and np.max(ndiff)>1:
            amax = np.argmax(ndiff)
            naidx[amax]
            mfit = meas[naidx[amax]+1:naidx[amax+1]-1]
        else:
            mfit = meas[~np.isnan(meas[:,0]),:]
        kf = KalmanFilter(initial_state_mean=np.concatenate((np.nanmean(mfit,axis=0),[0,0])),
                                                            n_dim_obs=2,n_dim_state=4)

        ### Aggressive filtering to detect outliers
        # transition matrices
        kf.transition_matrices = np.array([[1,0,1,0],[0,1,0,1],
                                           [0,0,1,0],[0,0,0,1]])

        # mask nan-values
        meas[np.isnan(meas)]=ma.masked

        kf.observation_covariance = np.array([[observation_error, observation_cov],
                                              [observation_cov, observation_error]])

        kf.transition_covariance = np.array([[pos_proc_error, pos_proc_xy_cov, pos_vel_proc_xx_cov, pos_vel_proc_xy_cov],
                                             [pos_proc_xy_cov, pos_proc_error, pos_vel_proc_xy_cov, pos_vel_proc_xx_cov],
                                             [pos_vel_proc_xx_cov, pos_vel_proc_xy_cov, vel_proc_error, vel_proc_xy_cov],
                                             [pos_vel_proc_xy_cov, pos_vel_proc_xx_cov, vel_proc_xy_cov, vel_proc_error]])/outl_cov_fact
        #pred_m,pred_cov=kf.em(mfit).smooth(meas)
        pred_m,pred_cov=kf.smooth(meas)
        pred_r = pred_m

        # Remove outliers from final filtering
        outl = np.sqrt(np.sum((meas-pred_m[:,:2])**2,axis=1))>outl_pos_thresh
        print(f"Outliers: {np.sum(outl)}/{len(outl)}")

        meas[outl,:]=ma.masked

        ### Final filtering 
        kf = KalmanFilter(initial_state_mean=np.concatenate((np.nanmean(mfit,axis=0),[0,0])),
                                                            n_dim_obs=2,n_dim_state=4)
        kf.transition_matrices = np.array([[1,0,1,0],[0,1,0,1],
                                           [0,0,1,0],[0,0,0,1]])

        kf.observation_covariance = np.array([[observation_error, observation_cov],
                                              [observation_cov, observation_error]])

        kf.transition_covariance = np.array([[pos_proc_error, pos_proc_xy_cov, pos_vel_proc_xx_cov, pos_vel_proc_xy_cov],
                                             [pos_proc_xy_cov, pos_proc_error, pos_vel_proc_xy_cov, pos_vel_proc_xx_cov],
                                             [pos_vel_proc_xx_cov, pos_vel_proc_xy_cov, vel_proc_error, vel_proc_xy_cov],
                                             [pos_vel_proc_xy_cov, pos_vel_proc_xx_cov, vel_proc_xy_cov, vel_proc_error]])

        pred_m,pred_cov=kf.smooth(meas)
    
        preds[:,ii,:] = pred_m[:,:2]
    return preds



Helper function to subtract two time series

In [ ]:
def ts_diff(ts1,ts2):
    t0 = max(ts1.t_start, ts2.t_start)
    te = min(ts1.t[-1],ts2.t[-1])
    t1,v1 = ts1.times_values_in_range(from_time=t0,to_time=te)
    t2,v2 = ts2.times_values_in_range(from_time=t0,to_time=te)
    return SampledTimeSeries(t=t1, v=np.interp(t1,t2,v2)-v1)

Helper function to calculate orientation of a line segment between two ponts

In [ ]:
def calc_angle(p1, p2):
    return 180/np.pi*np.arctan((p2[...,1]-p1[...,1])/(p2[...,0]-p1[...,0]))

In [ ]:
marg = 5
nmark=33
coords = ['x','y','z']

rec_dict = [{'wavfile': x} for x in ndf[ndf.gl_ver_idx.isin(trials)].wavfile.unique()]

for md in rec_dict:
    
    sigfile = md['wavfile']
    tsfile = ndf[ndf.wavfile==sigfile].sigfile.iloc[0]
    relpath = os.path.split(sigfile)[0]
    print(sigfile)
    
    # store needed time series for plot
    sr, w = sio.wavfile.read(os.path.join(dataroot,sigfile))
    tsl = ts_from_pickle(os.path.join(calcroot,tsfile))

    md['wav'] = w
    md['sr'] = sr
    
    try:
        md['f0'] = [ts for ts in tsl if ts.label=="barrel_f0"][0]
    except IndexError:
        md['f0'] = [ts for ts in tsl if ts.label=="external_f0"][0]
    md['ampl'] = [ts for ts in tsl if ts.label=="external_ampl"][0]
    try:
        md['mouth_dc'] = [ts for ts in tsl if ts.label=="mouth_dc"][0]
        md['reed_dc'] = [ts for ts in tsl if ts.label=="reed_dc"][0]
    except IndexError:
        print("No musician parameter")
    
    df = ndf[ndf.sigfile==tsfile]
    print(df.gl_ver_idx.unique())
    md['notes'] = df
    md['versions'] = df.gl_ver_idx.unique()

    
    # gather clarinet video data (front and side views)
    for view in ['front','side']:
        video_file_cell = videodf.loc[(videodf['Video file'].str.lower().str.match(f'.*{view}'))&
                                (videodf['Signal file']==sigfile),'Video file']
        video_file = video_file_cell.iloc[0]
        marker_file = os.path.splitext(video_file.strip())[0]+'.json'
        print(marker_file)
        with open(dataroot+'/'+relpath+'/'+marker_file,'r') as f:
            data=json.load(f)
            
        delay = videodf.loc[video_file_cell.index, 'Video to signal delay'].values[0]
        
        # rotate marker coordinates if needed
        ang = float(videodf.loc[video_file_cell.index,'Rotate'])/180*np.pi
        rot_mx=np.array([[np.cos(ang),np.sin(ang)],[-np.sin(ang),np.cos(ang)]])
        ids, time, vals, dims = parse_aruco(data)
        vals_rot = rot_mx.dot(vals.transpose((0,2,1))).transpose((1,2,0))
        
        # Select only interesting markers
        idnbrs = videodf.loc[video_file_cell.index, videodf.columns[videodf.columns.str.contains('marker')]].values[0]
        idi = np.asarray([np.where(ids==idnbr)[0][0] for idnbr in idnbrs])
        vals = vals_rot[:,idi,:]

        # select region of interest
        tmax = np.max(md['ampl'].t)
        idx = (time>-delay-marg)&(time<tmax-delay+marg)
        time = time[idx]
        vals = vals[idx]

        # Kalman filter fills in undetected markers in certain frames
        vals = kfilter(vals)
        print(vals.shape)

        # calculate clarinet angle
        cdiff = np.diff(vals,axis=1)[:,0,:]
        ang = -np.arctan(cdiff[:,0]/cdiff[:,1])/np.pi*180
        leng = np.sqrt(np.sum(cdiff**2,axis=1))
        
        # subtract video delay and generate timeseries
        time += delay
        md[f'{view}_angle']=SampledTimeSeries(t=time, v=ang)
        
        # POSE data
        try:            
            pose_file = os.path.splitext(video_file.strip())[0]+'_Pose.pickle'

            with open(dataroot+'/'+relpath+'/'+pose_file,'rb') as f:
                data=pickle.load(f)
             
            pose_angles = np.ones((len(data),5))*np.nan
            times = np.zeros(len(data))

            rad_to_deg = 180/np.pi
            
            # Markers to tensor
            vals = np.ones((len(data),nmark,len(coords)))*np.nan
            times = np.ones((len(data)))*np.nan
            for ti in range(len(data)):
                times[ti] = data[ti]['time']
                if 'landmarks' in data[ti]:
                    for ii in data[ti]['landmarks']:
                        for jj,coord in enumerate(coords):
                            try:
                                vals[ti,ii,jj] = data[ti]['landmarks'][ii][coord]
                            except IndexError:
                                pass
                            
            idx = (times>-delay-marg)&(times<tmax-delay+marg)
            # filter markers (only x-y coords)
            midx = np.array([6,8])
            fvals = kfilter(vals[idx,:,:2][:,midx,:],observation_error=15)

               
            #pose_angles = pose_angles[idx,:]
            times += delay
            md['head_angle'] = SampledTimeSeries(t=times[idx], v=calc_angle(fvals[:,0,:],fvals[:,1,:]))
            #md['mouth_hand_angle'] = SampledTimeSeries(t=times[idx], v=fvals[idx,4])
            #md['torso_angle'] = SampledTimeSeries(t=times[idx], v=pose_angles[idx,3])
        except IOError:
            print('Pose data note found')
    try:
        # gather mouthpiece video data 
        video_file_cell = videodf.loc[(videodf['Video file'].str.lower().str.match(f'.*mouthpiece'))&
                                (videodf['Signal file']==sigfile),'Video file']
        video_file = video_file_cell.iloc[0]
        marker_file = os.path.splitext(video_file.strip())[0]+'.json'
        print(marker_file)

        with open(dataroot+'/'+relpath+'/'+marker_file,'r') as f:
            data = json.load(f)

        time = np.array([x['time'] for x in data])
        vals = np.array([x['area'] for x in data])
        plt.figure()
        plt.plot(time,vals)
        plt.title(marker_file)
        plt.axvline(-videodf.loc[video_file_cell.index, 'Video to signal delay'].values[0],color='r')
        # subtract video delay and generate timeseries
        time += videodf.loc[video_file_cell.index, 'Video to signal delay'].values[0]
        
        t = np.arange(0,md['ampl'].t[-1],np.median(np.diff(time)))
        idx = (time>-5)&(time<md['ampl'].t[-1]+5)
        v = np.interp(t,time[idx],vals[idx])
        md['lips']=SampledTimeSeries(t=t, v=v)

    except IndexError:
        print("No mouthpiece video data found")


    # Calculate angle from vertical
    a1 = md['front_angle']
    a2 = md['side_angle']
    tmin = max(np.min(a1.t),np.min(a2.t))
    tmax = min(np.max(a1.t),np.max(a2.t))
    dt = min(a1.dt,a2.dt)
    t = np.arange(tmin,tmax,dt)

    md['angle'] = SampledTimeSeries(t=t,v=np.sqrt(a1[t]**2+a2[t]**2))
    md['cl_head_angle'] = ts_diff(md['side_angle'],md['head_angle']*-1)
    


# Plot figure 8 for article

In [ ]:
tslabels = [k for k,v in rec_dict[0].items() if type(v) is SampledTimeSeries]

marg=1
labels = {"f0":"Freq. (Hz)",
          "ampl":"RMS Ampl.\n(Pa)",
          "mouth_dc":"Blowing P.\n(Pa)",
          "reed_dc": "Reed pos.\n(norm)",
          "cl_head_angle": "Angle\n(deg)",
          "lips": "MP covering\n(Px)"}
fig,ax = plt.subplots(len(labels),sharex=True)

for d in rec_dict:
    for v in d['versions']:

        n = ndf[ndf.gl_ver_idx==v]
        
        for axi, lab in zip(ax,labels):
            tslab = lab
            try:
                ats = d[tslab]
                if tslab in ['reed_dc']:
                    vmax = ats.percentile(95)
                    vmin = ats.percentile(5)
                    ats = (ats-vmin)/(vmax-vmin)
                if tslab == 'ampl':
                    axi.set_yscale('log')
                if tslab == 'f0':
                    ats = ats.apply(sig.medfilt)
            except KeyError:
                axi.plot(0,0)
                continue
            #ats.apply(np.log10).plot()
            axi.set_ylabel(labels[tslab])

            tt,vv=ats.times_values_in_range(n.iloc[0].start-marg,n.iloc[0].start)
            tb = (tt-n.iloc[0].start).tolist()
            vb = vv.tolist()
            beats = []

            cur_beat=0
            for (nidx, note1), (nidx, note2) in zip(n.iterrows(),n.shift(-1).iterrows()):
                #plt.axvspan(note.start, note.end,alpha=.3)
                #if np.isnan(note2.start):
                end_beat = note1.duration+note1.start_beat
                if note2.start_beat is not None and end_beat>= note2.start_beat:
                    end = note2.start
                else:
                    end = note1.end
                tn, vn = ats.times_values_in_range(note1.start,end)

                tb.extend(np.linspace(note1.start_beat,end_beat,len(vn)))
                vb.extend(vn)

                if note2.start_beat is not None and end_beat < note2.start_beat:
                    tn, vn = ats.times_values_in_range(end,note2.start)
                    tb.extend(np.linspace(end_beat,note2.start_beat,len(vn)))
                    vb.extend(vn)
            tn, vn = ats.times_values_in_range(end,end+marg)
            tb.extend(tn-end+end_beat)
            vb.extend(vn)
            axi.plot(tb,(vb))
            

        print(n.iloc[0].to_dict())
        beats.append(cur_beat)



for bs,be in zip(n.start_beat, n.start_beat+n.duration):
    #print(b)
    for axi in ax:
        axi.axvline(bs,alpha=.3,color='r')
        axi.axvline(be,alpha=.3,color='m')

#ax[1].set_yscale('log')
#ax[0].set_ylabel('Frequency (Hz)')
#ax[1].set_ylabel('Amplitude (mPa)')
#ax[2].set_ylabel('Mouth p (Pa)')
#ax[3].set_ylabel('Reed (0-1)')

ax[-1].set_xlabel('Beat number')
ax[0].legend(['Player A (take 1)','Player A (take 2)','Player B (take 1)','Player B (take 2)'],loc='upper left',bbox_to_anchor=(1,1))
fig.align_ylabels()
plt.tight_layout()
plt.subplots_adjust(hspace=0)
